# **Milestone 3 - Data Validation**

- **Name:** Thaliban Allaam
- **Batch:** HCK - 025

**Objective:**
- To validate the processed data which is the output of the automation process from Apache Airflow.

# **1. Create Data Context**

The primary entry point for a Great Expectations deployment.

In [3]:
# Create a data context

from great_expectations.data_context import FileDataContext
context = FileDataContext.create(project_root_dir='./')

# **2. Connect to Data Source**

Connect to the data source that we want to validate.

In [4]:
# Give a name to a Datasource. This name must be unique between Datasources.
datasource_name = 'P2M3_ThalibanAllaam_data_clean'
datasource = context.sources.add_pandas(datasource_name)

# Give a name to a data asset
asset_name = 'superstore-sales'
path_to_data = 'compose_file\data\P2M3_ThalibanAllaam_data_clean.csv'
asset = datasource.add_csv_asset(asset_name, filepath_or_buffer=path_to_data)

# Build batch request
batch_request = asset.build_batch_request()

# **3. Create Expectation Suite**

The expectation suite is a collection of things/expectations that we want to verify/validate for the data.

In [5]:
# Creat an expectation suite
expectation_suite_name = 'expectation-superstore-dataset'
context.add_or_update_expectation_suite(expectation_suite_name)

# Create a validator using above expectation suite
validator = context.get_validator(
    batch_request = batch_request,
    expectation_suite_name = expectation_suite_name
)

# Check the validator
validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,state,postal_code,region,product_id,category,sub_category,product_name,sales
0,1,CA-2017-152156,2017-11-08,2017-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,2017-11-08,2017-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,2017-06-12,2017-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,2016-10-11,2016-10-18,Standard Class,SO-20335,Sean O Donnel,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,2016-10-11,2016-10-18,Standard Class,SO-20335,Sean O Donnel,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold N Roll Cart System,22.3680


## **Expectation 1**

We want the all the data in the`row_id` column to be unique to make sure there are no duplicated data in the dataset.

In [6]:
# Expectation 1 : Column `row_id` must be unique
validator.expect_column_values_to_be_unique('row_id')

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 9789,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

## **Expectation 2**

We want the value of sales from each order to be above 0 and below 100.000 to make sure there are no negative values and no sales above $ 100.000 which is the purchase limit.

In [7]:
# Expectation 2 : Column `sales` must be above $ 0 and below $ 100000
validator.expect_column_values_to_be_between(
    column='sales', min_value=0, max_value=100000
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 9789,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

## **Expectation 3**

We want to make sure that `customer_id` column exists so that we can trace back the purchases to the people who bought the items.

In [8]:
# Expectation 3 : Column `order_id` must exist

validator.expect_column_to_exist(column='customer_id')

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

{
  "success": true,
  "result": {},
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

## **Expectation 4**

We want to make sure that the `sales` column has a numerical data type so that the data can be processed accordingly.

In [9]:
# Expectation 4 : Column `sales` must in form of integer or float

validator.expect_column_values_to_be_in_type_list('sales', ['integer', 'float'])

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": "float64"
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

## **Expectation 5**

We want to make sure that every shipping option is available in the dataset. This is important so that we can compare the sales based on the shipping options.

In [10]:
# Expectation 5: All unique values ​​of ship_mode must be in the set.

validator.expect_column_distinct_values_to_be_in_set("ship_mode", ['Second Class', 'Standard Class', 'First Class', 'Same Day'])

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": [
      "First Class",
      "Same Day",
      "Second Class",
      "Standard Class"
    ],
    "details": {
      "value_counts": [
        {
          "value": "First Class",
          "count": 1501
        },
        {
          "value": "Same Day",
          "count": 538
        },
        {
          "value": "Second Class",
          "count": 1901
        },
        {
          "value": "Standard Class",
          "count": 5849
        }
      ]
    }
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

## **Expectation 6**

We want to make sure that the length of `order_id` must be between 6 and 15 so that the id is not too long, but also not too short. This is done to allow easier storage and identification process.

In [11]:
# Expectation 6: Order_ID length must be between 6 and 15
validator.expect_column_value_lengths_to_be_between(
    column="order_id",
    min_value=6,
    max_value=15
)

Calculating Metrics:   0%|          | 0/9 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 9789,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

## **Expectation 7**

We want to make sure that there are no missing values in sales to avoid error during further processing. Missing values in the sales column would be bad since sales is the main data of the dataset. If the sales value is missing, therefore the data is basically useless.

In [12]:
# Expectation 7: Sales must not be null
validator.expect_column_values_to_not_be_null("sales")

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 9789,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

After all the expectations are defined, we save the expectation suite.

In [13]:
# Save into Expectation Suite

validator.save_expectation_suite(discard_failed_expectations=False)

# **4. Create Checkpoint**

In [14]:
# Create a checkpoint

checkpoint_1 = context.add_or_update_checkpoint(
    name = 'checkpoint_1',
    validator = validator,
)

In [15]:
# Run a checkpoint

checkpoint_result = checkpoint_1.run()

Calculating Metrics:   0%|          | 0/29 [00:00<?, ?it/s]

# **5. Build Data Docs**

In [16]:
# Build data docs

context.build_data_docs()

{'local_site': 'file://c:\\Users\\allaa\\Desktop\\Hacktiv8\\P2W2\\p2-ftds025-hck-m3-thalibanallaam\\gx\\uncommitted/data_docs/local_site/index.html'}